In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('movie_data_no_blanks.csv')

In [ ]:
df.head()

,sentence,label
0,They do not!,not!
1,They do to!,to!
2,I hope so.,so.
3,She okay?,okay?
4,Let's go.,go.


In [ ]:
import torch

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-distilroberta-v1')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from typing import List, Tuple, Dict, Union

In [ ]:
# From A2

def get_st_embeddings(
    sentences: List[str],
    st_model: SentenceTransformer,
    batch_size: int = 32,
    device: str = "cpu",
):
    """
    Compute the sentence embedding using the Sentence Transformer model.

    Inputs:
    - sentence: The input sentence
    - st_model: SentenceTransformer model
    - batch_size: Encode in batches to avoid memory issues in case multiple sentences are passed

    Returns:
    torch.Tensor: The sentence embedding of shape [d,] (when only 1 sentence) or [n, d] where n is the number of sentences and d is the embedding dimension
    """

    st_model.to(device)
    sentence_embeddings = None

    for i in range(0, len(sentences), batch_size):
        batch_sentences = sentences[i : i + batch_size]
        batch_embeddings = st_model.encode(batch_sentences, convert_to_tensor=True)
        if sentence_embeddings is None:
            sentence_embeddings = batch_embeddings
        else:
            sentence_embeddings = torch.cat(
                [sentence_embeddings, batch_embeddings], dim=0
            )

    return sentence_embeddings.to("cpu")

In [ ]:
X_train_st = get_st_embeddings(df["sentence"].values, model, device=device, batch_size=256)

In [ ]:
from google.colab import files

In [ ]:
torch.save(X_train_st, "X_train_st.pt")
files.download('X_train_st.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
X_label_st = get_st_embeddings(df["label"].values, model, device=device, batch_size=256)

In [ ]:
torch.save(X_label_st, "X_label_st.pt")
files.download('X_label_st.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>